## Add channel files based on the EEG with fixes.

This notebook replaces the `channels.tsv` files with a manually generated set for
Sternberg Working Memory dataset.


In [1]:
import os
import json
import datetime
import pandas as pd
from hed.tools import get_file_list, get_value_dict, HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/SternbergWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
log_name = 'sternberg_10_fix_channels_log'
sampling_rate_file = os.path.realpath(os.path.join(bids_root_path, 'code/samplingRates.tsv'))
channel_file = os.path.realpath(os.path.join(bids_root_path, "code/channelsOriginal.json"))

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Load the channels file
with open(channel_file, 'r') as fp:
    channel_info = json.load(fp)

# Construct the lists of the EEG files
eeg_files = get_file_list(bids_root_path, extensions=[".set"], name_suffix="_eeg", exclude_dirs=exclude_dirs)

eeg_json_template = {
  "TaskName": "WorkingMemory",
  "EEGReference": "common",
  "RecordingType": "continuous",
  "RecordingDuration": 617.984,
  "SamplingFrequency": 500,
  "EEGChannelCount": 69,
  "EOGChannelCount": 2,
  "PowerLineFrequency": 60,
  "SoftwareFilters": "n/a"
}

# Get the sample rate dictionary
sampling_dict = get_value_dict(sampling_rate_file)
channel_names = ['FP1', 'FPZ', 'FP2', 'AF7', 'AF3', 'AFZ', 'AF4', 'AF8',
            'F9', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'F10',
            'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6',
            'FT8', 'FT10', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6',
            'T8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10',
            'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8',
            'PO9', 'PO7', 'PO3', 'POZ', 'PO4', 'PO8', 'PO10',
            'O1','OZ', 'O2', 'LEYE', 'REYE']

chan_dict = {'REYE': ('REYE', 'EOG', 'microV'),
             'LEYE': ('LEYE', 'EOG', 'microV')
             }

coordsystem_template = {"EEGCoordinateUnits": "n/a",
                        "EEGCoordinateSystem":"CTF",
                        "EEGCoordinateSystemDescription":"EEGLAB"}


for file_name in eeg_files:
    basename = os.path.basename(file_name)
    channels = channel_info[basename]
    logger.add(basename, f"EEG has {len(channels)} channels")
    sampling_key = basename[:-8].replace("WorkingMemory", "Experiment")
    sampling_rate = sampling_dict[sampling_key]
    data = {'name': channel_names}
    df = pd.DataFrame(data)
    df[['type', 'units']] = 'n/a'
    names = df['name']
    for index, name in names.iteritems():
        row = chan_dict.get(name, None)
        if not row:
            row = (name, 'EEG', 'microV')
        df.loc[index, 'name'] = row[0]
        df.loc[index, 'type'] = row[1]
        df.loc[index, 'units'] = row[2]
    EEG = sum(df['type'].map(str) == 'EEG')
    EOG = sum(df['type'].map(str) == 'EOG')
    OTHER = sum(df['type'].map(str) == 'OTHER')
    MISC = sum(df['type'].map(str) == 'MISC')
    logger.add(basename, f"{EEG} EEG channels, {EOG} EOG channels, {OTHER} OTHER channels {MISC} MISC channels")
    file_new = file_name[:-7] + "channels.tsv"
    df.to_csv(file_new, sep='\t', index=False)
    logger.add(basename, f"Saved as {os.path.basename(file_new)}")
    eeg_json_file = file_name[:-4] + ".json"
    with open(eeg_json_file, 'r') as fp:
        eeg_json_input = json.load(fp)
    eeg_json = {}
    for key, value in eeg_json_template.items():
        eeg_json[key] = value
    eeg_json["RecordingDuration"] = eeg_json_input["RecordingDuration"]
    eeg_json["SamplingFrequency"] = sampling_rate
    json_new = file_name[:-7] + "coordsystem.json"
    with open(json_new, 'w') as fp:
        json.dump(coordsystem_template, fp)
    logger.add(basename, f"Saving the {os.path.basename(json_new)} coordinate system file")

    with open(eeg_json_file, 'w') as fp:
        json.dump(eeg_json, fp, indent=4)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
sternberg_10_fix_channels_log: Level None
sub-001_ses-01_task-WorkingMemory_run-1_eeg.set:
	[ EEG has 71 channels]
	[ 69 EEG channels, 2 EOG channels, 0 OTHER channels 0 MISC channels]
	[ Saved as sub-001_ses-01_task-WorkingMemory_run-1_channels.tsv]
	[ Saving the sub-001_ses-01_task-WorkingMemory_run-1_coordsystem.json coordinate system file]
sub-001_ses-01_task-WorkingMemory_run-2_eeg.set:
	[ EEG has 71 channels]
	[ 69 EEG channels, 2 EOG channels, 0 OTHER channels 0 MISC channels]
	[ Saved as sub-001_ses-01_task-WorkingMemory_run-2_channels.tsv]
	[ Saving the sub-001_ses-01_task-WorkingMemory_run-2_coordsystem.json coordinate system file]
sub-001_ses-01_task-WorkingMemory_run-3_eeg.set:
	[ EEG has 71 channels]
	[ 69 EEG channels, 2 EOG channels, 0 OTHER channels 0 MISC channels]
	[ Saved as sub-001_ses-01_task-WorkingMemory_run-3_channels.tsv]
	[ Saving the sub-001_ses-01_task-WorkingMemory_run-3_coordsystem.json coordinate system file]
sub-001_ses-01_task-WorkingMemor